In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from glob import glob
from keras.utils import np_utils
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

import cv2
%matplotlib inline
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.listdir('/kaggle/input/plant-seedlings-classification')


In [ ]:
mainDir = "/kaggle/input/plant-seedlings-classification/"
os.listdir(mainDir + "train/")

In [ ]:
plt.imshow(img)

In [ ]:
img = cv2.imread(mainDir + "train/Scentless Mayweed/" + 
                 os.listdir(mainDir + "train/Scentless Mayweed/")[0])

im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(im_rgb)

In [ ]:
im_rgb.shape

### Think 

## Directory Structure 
--- mainDir
------ SeedlingType
--------- Imagename

------ EDA

Process 1 : Read all images and save and npy file 
Process 2 : Use Batchgenerator 

---- Batch Generator 

1) It takes a batch of images
2) It can apply augmentations 

---- CNN Model

### Exploratory Data Analysis

In [ ]:
Imgscale = 80
randomSeed = 2021

In [ ]:
### Reading images 


path2images = mainDir + "train/" + "*/*.png"

allImages = glob(path2images)

imgArray = []
imgLabel = []

totalImg = len(allImages)
for i,eachImage in enumerate(allImages):
    print(f"{i} / {totalImg} ",end = "\r")
    imgArray.append(cv2.resize(cv2.imread(eachImage),(Imgscale,Imgscale)))
    imgLabel.append(eachImage.split("/")[-2])
    


In [ ]:
# j = 0
# for eachImage in allImages:
#     print(eachImage.split("/")[-2])
    
#     j += 1
    
#     if j > 2:
#         break

In [ ]:
type(imgArray)

In [ ]:
imgArray = np.asarray(imgArray)
imgLabel = pd.DataFrame(imgLabel)

type(imgArray)

In [ ]:
imgLabel.head()

In [ ]:
len(imgArray[:3])

In [ ]:
### Let's see some examples 
plt.figure(figsize=(10, 10))

for j,img in enumerate(imgArray[:8]):
    plt.subplot(4, 2, j+1)
    im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(im_rgb)
    plt.axis("off")



In [ ]:
### Let's see some examples 
plt.figure(figsize=(10, 10))

for j,img in enumerate(imgArray[:1]):
    plt.subplot(4, 2, j+1)
    ### Gaussian Blur 
    imgBlur = cv2.GaussianBlur(img,(5,5),0)
    img_rgb = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2HSV)
    
    ### mask creation -- green color 
    
    greenLow = (25,40,50)
    greenUp = (75,255,255)
    mask = cv2.inRange(img_hsv,greenLow,greenUp)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11))
    mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    
    
    bMask = mask > 0 
    
    clear = np.zeros_like(img,np.uint8)
    clear[bMask] = img[bMask]
    
    plt.subplot(2, 3, 1); plt.imshow(img)  # Show the original image
    plt.subplot(2, 3, 2); plt.imshow(imgBlur)  # Blur image
    plt.subplot(2, 3, 3); plt.imshow(img_hsv)  # HSV image
    plt.subplot(2, 3, 4); plt.imshow(mask)  # Mask
    plt.subplot(2, 3, 5); plt.imshow(bMask)  # Boolean mask
    plt.subplot(2, 3, 6); plt.imshow(clear)  # Image without background
    
    
    



In [ ]:
def imgCleaner(img):
    imgBlur = cv2.GaussianBlur(img,(5,5),0)
    img_rgb = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2HSV)
    ### mask creation -- green color 
    greenLow = (25,40,50)
    greenUp = (75,255,255)
    mask = cv2.inRange(img_hsv,greenLow,greenUp)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11))
    mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE,kernel)
    bMask = mask > 0 
    clear = np.zeros_like(img,np.uint8)
    clear[bMask] = img[bMask]
    return clear

In [ ]:
### Let's see some examples 
plt.figure(figsize=(10, 10))

for j,img in enumerate(imgArray[:8]):
    plt.subplot(4, 2, j+1)
    im_cleaned = imgCleaner(img)
    plt.imshow(im_cleaned)
    plt.axis("off")



In [ ]:
imgArrayCleaned = []
for j,img in enumerate(imgArray):
    im_cleaned = imgCleaner(img)
    imgArrayCleaned.append(im_cleaned)
    
    
imgArrayCleaned = np.asarray(imgArrayCleaned)

In [ ]:
imgArrayCleaned.shape,imgArray.shape

In [ ]:
imgArrayCleaned = imgArrayCleaned / 255


#### Labels

In [ ]:
imgLabel[0].value_counts().plot(kind = 'bar')

In [ ]:
### Steps to encode labels from text to one hot encoding
## Step 1 -- Label encoding 

labEnc = preprocessing.LabelEncoder()
imgLabelEncode = labEnc.fit_transform(imgLabel[0])
print("Classes: " + str(labEnc.classes_))
num_clases = len(labEnc.classes_)
print(f"Number of classes are {num_clases}")

### Step 2 -- One Hot encoding using keras np_utils

imgLabelOhe = np_utils.to_categorical(imgLabelEncode)
imgLabelOhe[1]

### CNN Model

In [ ]:
### Data Split 
trainImgX, testImgX,trainImgY,testImgY = train_test_split(imgArrayCleaned,imgLabelOhe,test_size = .15,
                                                         random_state = randomSeed,stratify = imgLabelOhe)


### Datagenerator

In [ ]:
dataGen = ImageDataGenerator(rotation_range=180,zoom_range=.1,width_shift_range=.1,height_shift_range = .1,
                             horizontal_flip=True,vertical_flip=True)

In [ ]:
dataGen.fit(trainImgX)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,GlobalMaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam


In [ ]:
np.random.seed(randomSeed)  # Fix seed


In [ ]:
def ModelCNN_Small(num_clases):

    model = Sequential()
    model.add(Conv2D(32, (5,5), padding='same', activation="relu", input_shape=(128, 128, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    # model.add(layers.Dropout(0.2))
    model.add(Conv2D(64, (3,3), padding='same', activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    # model.add(layers.Dropout(0.3))
    model.add(Conv2D(64, (3, 3), padding='same', activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    # model.add(layers.Dropout(0.4))
    model.add(Conv2D(128, (3, 3), padding='same', activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
  
    model.add(GlobalMaxPooling2D())
    ## ANN 
    model.add(Dense(64, activation="relu"))
    

    model.add(Dense(num_clases, activation="softmax"))

    optimizer = Adam(learning_rate=0.001,beta_1=0.9,epsilon=1e-07,amsgrad=False,name='Adam')
    model.compile(optimizer = optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
def CNNModel(num_clases):
    model = Sequential()

    model.add(Conv2D(filters=64, kernel_size=(5, 5), input_shape=(Imgscale, Imgscale, 3), activation='relu'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization(axis=3))
    model.add(Dropout(0.1))

    model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization(axis=3))
    model.add(Dropout(0.1))

    model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
    model.add(BatchNormalization(axis=3))
    model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(BatchNormalization(axis=3))
    model.add(Dropout(0.1))

    model.add(Flatten())

    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(num_clases, activation='softmax'))

    model.summary()

    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger,EarlyStopping
# learning rate reduction
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.4, 
                                            min_lr=0.00001)


early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=10)

# checkpoints
filepath="./weights.best_{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', 
                             verbose=1, save_best_only=True, mode='auto')
filepath="./weights.last_auto4.hdf5"
checkpoint_all = ModelCheckpoint(filepath, monitor='val_loss', 
                                 verbose=1, save_best_only=False, mode='auto')


# all callbacks
callbacks_list = [checkpoint, learning_rate_reduction,early_stopping]

In [ ]:
trainGen = dataGen.flow(trainImgX, trainImgY, batch_size=64)
validGen = dataGen.flow(testImgX, testImgY, batch_size=64)

In [ ]:
model = ModelCNN_Small(num_clases)

In [ ]:
hist = model.fit_generator(trainGen, 
                           epochs=40, validation_data=validGen, 
                           steps_per_epoch=trainImgX.shape[0]//64,callbacks=callbacks_list )

Training Graphs

In [ ]:
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
# plt.ylim([0.5, 1])
plt.legend(loc='lower right');

In [ ]:
plt.plot(hist.history['accuracy'], label='accuracy')
plt.plot(hist.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
plt.legend(loc='lower right');

### Model Evaluation

In [ ]:
print(model.evaluate(testImgX, testImgY))  # Evaluate on test set


Confusion Matrix

In [ ]:
# Predict the values from the validation dataset
Y_pred = model.predict(testImgX)
# Convert predictions classes to one hot vectors 
result = np.argmax(Y_pred, axis=1)
# Convert validation observations to one hot vectors
Y_true = np.argmax(testImgY, axis=1)

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(Y_true, result)

df_cm = pd.DataFrame(conf_mat, index = [i for i in range(0, 12)],
                  columns = [i for i in range(0, 12)])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True, fmt='g');

Some Examples

In [ ]:
X_test  = testImgX
y_test = testImgY
plt.figure(figsize=(2,2))
plt.imshow(X_test[3],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[3].reshape(1,Imgscale,Imgscale,3))))
print('True Label', np.argmax(y_test[3]))

plt.figure(figsize=(2,2))
plt.imshow(X_test[2],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[2].reshape(1,Imgscale,Imgscale,3))))
print('True Label', np.argmax(y_test[2]))

plt.figure(figsize=(2,2))
plt.imshow(X_test[33],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[33].reshape(1,Imgscale,Imgscale,3))))
print('True Label', np.argmax(y_test[33]))

plt.figure(figsize=(2,2))
plt.imshow(X_test[59],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[59].reshape(1,Imgscale,Imgscale,3))))
print('True Label', np.argmax(y_test[59]))

plt.figure(figsize=(2,2))
plt.imshow(X_test[36],cmap="gray")
plt.show()
print('Predicted Label', np.argmax(model.predict(X_test[36].reshape(1,Imgscale,Imgscale,3))))
print('True Label', np.argmax(y_test[36]))

#### Visualisation of Feature Maps ( Activation Maps)

Load the trained model ( We need to create model and then just load the weights)

In [ ]:
modelTrained = ModelCNN_Small(num_clases=num_clases)

In [ ]:
modelTrained.load_weights("./weights.best_36-0.36.hdf5")


In [ ]:

# modelTrained.load_weights("../input/trainedmodelplantseed/weights.best_01-25.64.hdf5")


In [ ]:
plt.imshow(cv2.resize(im_rgb,(128,128)))

In [ ]:
layer_outputs = [layer.output for layer in modelTrained.layers[1:7]]
activation_model = Model(inputs=modelTrained.input,outputs=layer_outputs)
activations = activation_model.predict(cv2.resize(im_rgb,(128,128)).reshape(1,128,128,3))
layer_names = []
for layer in modelTrained.layers[1:7]:
    layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot
    

images_per_row = 16
for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
    n_features = layer_activation.shape[-1] # Number of features in the feature map
    size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    for col in range(n_cols): # Tiles each filter into a big horizontal grid
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')


In [ ]:
# layer_outputs = [layer.output for layer in model.layers[1:7]]
# activation_model = Model(inputs=model.input,outputs=layer_outputs)
# activations = activation_model.predict(testImgX[3].reshape(1,scale,scale,3))


In [ ]:
# layer_names = []
# for layer in model.layers[1:7]:
#     layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot
    

# images_per_row = 16
# for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
#     n_features = layer_activation.shape[-1] # Number of features in the feature map
#     size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
#     n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
#     display_grid = np.zeros((size * n_cols, images_per_row * size))
#     for col in range(n_cols): # Tiles each filter into a big horizontal grid
#         for row in range(images_per_row):
#             channel_image = layer_activation[0,
#                                              :, :,
#                                              col * images_per_row + row]
#             channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
#             channel_image /= channel_image.std()
#             channel_image *= 64
#             channel_image += 128
#             channel_image = np.clip(channel_image, 0, 255).astype('uint8')
#             display_grid[col * size : (col + 1) * size, # Displays the grid
#                          row * size : (row + 1) * size] = channel_image
#     scale = 1. / size
#     plt.figure(figsize=(scale * display_grid.shape[1],
#                         scale * display_grid.shape[0]))
#     plt.title(layer_name)
#     plt.grid(False)
#     plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
idg = ImageDataGenerator(
    rescale=1./255,
    #rotation_range=20, # You can uncomment these parameters to make you generator rotate & flip the images to put the train model in stricter conditions.
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2
)

In [ ]:
mainDir

In [ ]:
train_gen = idg.flow_from_directory(mainDir + 'train/',
                                                    target_size=(Imgscale, Imgscale),
                                                    subset='training',
                                                    class_mode='categorical',
                                                    batch_size=64,
                                                    shuffle=True,
                                                    seed=1
                                                )

In [ ]:
val_gen = idg.flow_from_directory(mainDir + 'train/',
                                                   target_size=(Imgscale, Imgscale),                                                   
                                                   subset='validation',
                                                   class_mode='categorical',
                                                   batch_size=64,
                                                   shuffle=True,
                                                   seed=1
                                                )

In [ ]:
model2train = CNNModel(num_clases=num_clases)

In [ ]:
historyNew = model2train.fit_generator(train_gen,
          epochs=2,
        steps_per_epoch= 500,validation_steps = 100,
        validation_data=val_gen,verbose = 1)